# 로컬에서 훈련 하기
- https://www.kaggle.com/code/mitanshuchakrawarty/fine-tune-llm-for-text-summary

## 1. 환경 셋업

In [10]:
from dotenv import load_dotenv

import os

HF_TOKEN = os.getenv('HF_TOKEN')

!huggingface-cli login --token {HF_TOKEN}

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/ec2-user/SageMaker/.cache/token
Login successful


In [11]:
import os 
os.environ['TRANSFORMERS_CACHE'] = "/home/ec2-user/SageMaker/.cache" 
os.environ['HF_DATASETS_CACHE'] = "/home/ec2-user/SageMaker/.cache" 
os.environ['HF_HOME'] = "/home/ec2-user/SageMaker/.cache"

In [12]:
%store -r data_folder
%store -r train_data_json 
%store -r validation_data_json 
%store -r test_data_json 
%store -r full_train_data_json 
%store -r full_validation_data_json 
%store -r full_test_data_json


print("data_folder: ", data_folder)
print("train_data_json: ", train_data_json)
print("validation_data_json: ", validation_data_json)
print("test_data_json: ", test_data_json)
print("full_train_data_json: ", full_train_data_json)
print("full_validation_data_json: ", full_validation_data_json)
print("full_test_data_json: ", full_test_data_json)

data_folder:  ../data/naver-news-summarization-ko
train_data_json:  ../data/naver-news-summarization-ko/train/train_dataset.json
validation_data_json:  ../data/naver-news-summarization-ko/validation/validation_dataset.json
test_data_json:  ../data/naver-news-summarization-ko/test/test_dataset.json
full_train_data_json:  ../data/naver-news-summarization-ko/full_train/train_dataset.json
full_validation_data_json:  ../data/naver-news-summarization-ko/full_validation/validation_dataset.json
full_test_data_json:  ../data/naver-news-summarization-ko/full_test/test_dataset.json


In [13]:
import torch
import time
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from datasets import Dataset, load_dataset
from datasets import load_dataset, load_metric
from transformers import pipeline, set_seed
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import warnings
warnings.filterwarnings("ignore")

## 2. 베이스 모델 준비

In [14]:
model_id = "meta-llama/Meta-Llama-3-8B"
output_dir = "/home/ec2-user/SageMaker/models/llama-3-8b-naver-news"

### Config YAML 파일 생성

In [15]:
%%writefile local_llama_3_8b_fsdp_qlora.yaml
# script parameters
model_id:  "meta-llama/Meta-Llama-3-8B" # Hugging Face model id
###########################
# small samples for Debug
###########################
# train_dataset_path: "../data/naver-news-summarization-ko/train"                      # path to dataset
# validation_dataset_path: "../data/naver-news-summarization-ko/validation"                      # path to dataset
# test_dataset_path: "../data/naver-news-summarization-ko/test"                      # path to dataset
# per_device_train_batch_size: 1         # batch size per device during training
# per_device_eval_batch_size: 1          # batch size for evaluation
# gradient_accumulation_steps: 2         # number of steps before performing a backward/update pass
###########################
# large samples for evaluation
###########################
train_dataset_path: "../data/naver-news-summarization-ko/full_train"                      # path to dataset
validation_dataset_path: "../data/naver-news-summarization-ko/full_validation"                      # path to dataset
test_dataset_path: "../data/naver-news-summarization-ko/full_test"                      # path to dataset
per_device_train_batch_size: 16         # batch size per device during training
per_device_eval_batch_size: 1          # batch size for evaluation
gradient_accumulation_steps: 2         # number of steps before performing a backward/update pass
###########################
max_seq_len:  2048              # max sequence length for model and packing of the dataset
# training parameters
output_dir: "/home/ec2-user/SageMaker/models/llama-3-8b-naver-news" # Temporary output directory for model checkpoints
report_to: "tensorboard"               # report metrics to tensorboard
learning_rate: 0.0002                  # learning rate 2e-4
lr_scheduler_type: "constant"          # learning rate scheduler
num_train_epochs: 1                    # number of training epochs
optim: adamw_torch                     # use torch adamw optimizer
logging_steps: 10                      # log every 10 steps
save_strategy: epoch                   # save checkpoint every epoch
evaluation_strategy: epoch             # evaluate every epoch
max_grad_norm: 0.3                     # max gradient norm
warmup_ratio: 0.03                     # warmup ratio
bf16: true                             # use bfloat16 precision
tf32: true                             # use tf32 precision
gradient_checkpointing: true           # use gradient checkpointing to save memory
# FSDP parameters: https://huggingface.co/docs/transformers/main/en/fsdp
fsdp: "full_shard auto_wrap offload" # remove offload if enough GPU memory
fsdp_config:
  backward_prefetch: "backward_pre"
  forward_prefetch: "false"
  use_orig_params: "false"

Overwriting local_llama_3_8b_fsdp_qlora.yaml


## 3. 훈련 Script 실행

In [16]:
!ACCELERATE_USE_FSDP=1 FSDP_CPU_RAM_EFFICIENT_LOADING=1 torchrun --nproc_per_node=4 \
../scripts/local_run_fsdp_qlora.py \
--config local_llama_3_8b_fsdp_qlora.yaml

[2024-06-30 02:17:50,760] torch.distributed.run: [WARNING] 
[2024-06-30 02:17:50,760] torch.distributed.run: [WARNING] *****************************************
[2024-06-30 02:17:50,760] torch.distributed.run: [WARNING] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
[2024-06-30 02:17:50,760] torch.distributed.run: [WARNING] *****************************************
/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use

## 4. 베이스 모델과 훈련된 모델 머지

In [17]:
model_id, output_dir

('meta-llama/Meta-Llama-3-8B',
 '/home/ec2-user/SageMaker/models/llama-3-8b-naver-news')

### 모델 머지 및 로컬에 저장
- 약 2분 걸림

In [18]:
from peft import AutoPeftModelForCausalLM
import torch

# Load PEFT model on CPU

model = AutoPeftModelForCausalLM.from_pretrained(
    output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)  
# Merge LoRA and base model and save
merged_model = model.merge_and_unload()
merged_model.save_pretrained(output_dir,safe_serialization=True, max_shard_size="2GB")

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  2.00it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
model.device, merged_model.device

(device(type='cpu'), device(type='cpu'))

### 머지된 모델 로딩

In [20]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer 


# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  pretrained_model_name_or_path = output_dir,
  torch_dtype=torch.float16,
  quantization_config= {"load_in_4bit": True},
  device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.31s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
model.device

device(type='cuda', index=1)

## 5. 추론

### 테스트 데이터 셋 로딩

In [25]:
from datasets import load_dataset 
from random import randint

def get_message_from_dataset(sample_dataset_json_file):
    # Load our test dataset
    full_test_dataset = load_dataset("json", data_files=sample_dataset_json_file, split="train")

    # Test on sample 
    rand_idx = randint(0, len(full_test_dataset)-1)
    rand_idx = 75
    print("rand_idx: ", rand_idx)
    messages = full_test_dataset[rand_idx]["messages"][:2]
    # messages = test_dataset[rand_idx]["text"][:2]
    print("messages: \n", messages)

    return messages, full_test_dataset, rand_idx

messages, full_test_dataset, rand_idx = get_message_from_dataset(sample_dataset_json_file = full_test_data_json)    

rand_idx:  75
messages: 
 [{'content': 'You are an AI assistant specialized in news articles.Your role is to provide accurate summaries and insights in Korean. Please analyze the given text and provide concise, informative summaries that highlight the key goals and findings.', 'role': 'system'}, {'content': 'Please summarize the goals for journalist in this text:\n\n한국 건설사 수주 기대감 높아 성공시 제2 중동 붐 전망까지 삼성물산·현대건설 컨소시엄 10억달러 규모 터널공사 수주 이미지출처 연합뉴스 사우디아라비아의 실세 무함마드 빈살만 제1왕위 계승자 왕세자 가 사우디를 개방적이고 온건한 이슬람 국가로 바꾸겠다고 선언했다. 국제유가 상승으로 사우디 아라비아의 국부가 늘면서 대규모 인프라 투자로 이어지자 한국 건설사들의 수주 기대감이 높아지고 있다. 대표적인 것이 네옴시티 로 사우디 실권자인 무함마드 빈 살만 왕세자가 주도하는 이 프로젝트는 총 사업비만 무려 5000억달러 약 640조원 에 달하는 세계 최대 규모 인프라 사업이다. 이미 물밑에서 수주 경쟁이 치열한 이 사업은 한국 건설사들이 대규모 수주에 성공할 경우 제 2의 중동 붐 을 일으킬 것이란 전망까지 나온다. 1일 증권업계 및 건설업계에 의하면 사우디발 인프라 건설 훈풍으로 국내 건설사들의 수주 기대감이 커지고 있다. 사우디는 2016년 발표한 비전 2030 에 의해 국가전략을 수행하고 있는데 이는 석유에 의존해온 경제를 첨단 제조업 중심으로 전환하기 위한 프로젝트로 네옴시티도 이 중 하나다. 이미 사우디는 비공개로 입찰을 진행 국내에선 삼성물산과 현대건설이 컨소시엄을 구성해 네옴시티 더 라인 의 터널 공사를 수주

### 추론

In [26]:

def generate_response(messages, model, tokenizer, full_test_dataset, rand_idx):
    input_ids = tokenizer.apply_chat_template(messages,add_generation_prompt=True,return_tensors="pt").to(model.device)
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id= tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]

    print(f"**Query:**\n{full_test_dataset[rand_idx]['messages'][1]['content']}\n")
    # print(f"**Query:**\n{test_dataset[rand_idx]['text'][1]['content']}\n")
    # print(f"**Original Answer:**\n{test_dataset[rand_idx]['text'][2]['content']}\n")
    print(f"**Original Answer:**\n{full_test_dataset[rand_idx]['messages'][2]['content']}\n")
    print(f"**Generated Answer:**\n{tokenizer.decode(response,skip_special_tokens=True)}")

generate_response(messages, model, tokenizer, full_test_dataset, rand_idx)    



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


**Query:**
Please summarize the goals for journalist in this text:

한국 건설사 수주 기대감 높아 성공시 제2 중동 붐 전망까지 삼성물산·현대건설 컨소시엄 10억달러 규모 터널공사 수주 이미지출처 연합뉴스 사우디아라비아의 실세 무함마드 빈살만 제1왕위 계승자 왕세자 가 사우디를 개방적이고 온건한 이슬람 국가로 바꾸겠다고 선언했다. 국제유가 상승으로 사우디 아라비아의 국부가 늘면서 대규모 인프라 투자로 이어지자 한국 건설사들의 수주 기대감이 높아지고 있다. 대표적인 것이 네옴시티 로 사우디 실권자인 무함마드 빈 살만 왕세자가 주도하는 이 프로젝트는 총 사업비만 무려 5000억달러 약 640조원 에 달하는 세계 최대 규모 인프라 사업이다. 이미 물밑에서 수주 경쟁이 치열한 이 사업은 한국 건설사들이 대규모 수주에 성공할 경우 제 2의 중동 붐 을 일으킬 것이란 전망까지 나온다. 1일 증권업계 및 건설업계에 의하면 사우디발 인프라 건설 훈풍으로 국내 건설사들의 수주 기대감이 커지고 있다. 사우디는 2016년 발표한 비전 2030 에 의해 국가전략을 수행하고 있는데 이는 석유에 의존해온 경제를 첨단 제조업 중심으로 전환하기 위한 프로젝트로 네옴시티도 이 중 하나다. 이미 사우디는 비공개로 입찰을 진행 국내에선 삼성물산과 현대건설이 컨소시엄을 구성해 네옴시티 더 라인 의 터널 공사를 수주한 것으로 알려졌다. 수주액은 약 10억달러 약 1조3000억원 규모다. 미래 먹거리로 로봇 및 자율주행과 같은 스마트 시티 를 추진하고 있는 현대차도 수주 가능성이 높은 것으로 알려졌다. 이에삼성엔지니어링 현대건설 대우건설 GS건설 DL이앤씨 등 해외 인프라 건설 사업 대형 5개사는 주택을 제외한 신규 수주 달성치를 높여잡았다. 현재 발주시장 확대에 근거해 이들 5개사는 2023년에는 신규 수주 금액이 합산 32조원으로 20%이상 성장 가능할 것으로 전망된다. 최대규모 인프라 사업인 네옴시티를 제외하고서라도 향후 해외건설시장 규모는